# Outputs (and inputs, but no signing details)

As you saw in the transaction parsing notebook, an output of a bitcoin transaction is made of three fields:

* Value 
* Script Length
* Script

The **value** is the number of satoshis 'encumbered' by the **script** that is **script length** bytes long.

As we will will explore in this notebook, the script is not simply an address with an owner, but a small piece of code. To spend an 'unspent transacion output' (utxo), you have to provide information that, when combined with the script, evaluate to True, but that will be discussed later and in the notebook about inputs.

## Parsing an Output Script

The transaction we de-serialized in the first notebook had two output scripts:

In [1]:
output_script_1 = "76a914a3d89c53bb956f08917b44d113c6b2bcbe0c29b788ac"

In [2]:
print("Output Script 1: ", output_script_1)

Output Script 1:  76a914a3d89c53bb956f08917b44d113c6b2bcbe0c29b788ac


In [3]:
output_script_2 = "76a91408338e1d5e26db3fce21b011795b1c3c8a5a5d0788ac"

In [4]:
print("Output Script 2: ", output_script_2)

Output Script 2:  76a91408338e1d5e26db3fce21b011795b1c3c8a5a5d0788ac


Notice that both scripts start with 76a914 and end with 88ac, this isn't a coincidence.

### OPCODES

*Script* is the rudimentary language that bitcoin uses to express the logic used to encumber an output. The scripts above are still hex-encoded bytes, so we need to decode those bytes into 'opcodes' which are the individual instructions interpreted by the bitcoin software.

The only true reference for available opcodes and their encoding is the actual bitcoind code:
https://github.com/bitcoin/bitcoin/blob/0.16/src/script/script.h#L47-L192

but this link does have descriptions that are a useful reference:
https://en.bitcoin.it/wiki/Script#Opcodes

For The first output script:
The first byte of both scripts is 76 which is OP_DUP
The second byte of both scripts is a9 which is OP_HASH160
The third byte of both scripts is 14 which is an instruction to push 0x14 (20) bytes to the stack.


In [5]:
print('Those bytes are: ', output_script_1[6:46])

Those bytes are:  a3d89c53bb956f08917b44d113c6b2bcbe0c29b7


The second to last byte is 88 which is OP_EQUALVERIFY
The last byte is ac which is OP_CHECKSIG

The second output script is the same except for the 20 bytes in the middle

In [6]:
print('which are: ', output_script_2[6:46])

which are:  08338e1d5e26db3fce21b011795b1c3c8a5a5d07


The python-bitcoinlib module has a script parser built-in so we don't need build one of our own.

In [7]:
from bitcoin.core import CScript

In [8]:
parsed_script_1 = CScript.fromhex(output_script_1)
print("Parsed Script 1: ")
parsed_script_1

Parsed Script 1: 


CScript([OP_DUP, OP_HASH160, x('a3d89c53bb956f08917b44d113c6b2bcbe0c29b7'), OP_EQUALVERIFY, OP_CHECKSIG])

## Script Evaluation

Bitcoin script is stack-based reverse polish notation.

This means that instead of writing 3 + 4 to do addition, you would do 3 4 +. 

You push 3 to the stack, then 4 to the stack, then use the + operator on the top two values of the stack.

In bitcoin transactions, you the output is the second half on an equation written in script. The input script (ScriptSig) is the first part of the equation so that when they are combined they evaluate to true.

For these outputs, we have OP_DUP, OP_HASH160, bytes, OP_EQUALVERIFY, OP_CHECKSIG in that order. This is what is known as a P2PKH (pay to public key hash).

Go to https://bitcoin.org/en/developer-guide#p2pkh-script-validation for a visualization on how a p2pkh script is evaluated.

After that, head back to https://blockchain.info/tx/9021b49d445c719106c95d561b9c3fac7bcb3650db67684a9226cd7fa1e1c1a0 and find the input of the transaction where one of the outputs is spent. The format should be (SIG, PUBKEY)

### ASIDE: Where exactly is that bitcoin address?

When you give someone a bitcoin address, like the 1FwLde9A8xyiboJkmjpBnVUYi1DTbXi8yf in the output of this transaction, where does that go? The bytes field in the middle of the P2PKH script is that address, decoded into hex.

The bitcoin address is encoded in base58Check encoding

TODO: base58check notebook.

In [9]:
from bitcoin.base58 import CBase58Data
from bitcoin.core import x

In [10]:
CBase58Data.from_bytes(x(output_script_1[6:46]),0)

CBase58Data('1FwLde9A8xyiboJkmjpBnVUYi1DTbXi8yf')

In [11]:
CBase58Data.from_bytes(x(output_script_2[6:46]),0)

CBase58Data('1kNAebmDfKcAfEEC2cRgyVFw5jRSjsAyk')

## Other common output formats:

From https://bitcoin.org/en/developer-guide#standard-transactions

### P2PK

Pubkey Script: ```pubkey OP_CHECKSIG```    
Signature script: ```sig```

### OP_RETURN

Pubkey Script: ```OP_RETURN (0 to 40 bytes of data)```

This is a way to put arbitrary bytes in the blockchain. The value of this type of output should be zero.
You still have to pay a fee for this transaction, so you need an input, and probably an additional output for the value left over after the fee is paid.

### Old Multisig

Pubkey script: ```M (A pubkey) (B pubkey) (C pubkey...) N OP_CHECKMULTISIG```  
Signature script: ```OP_0 A_sig (B sig) (C sig...)```
    
The OP_0 at the beginning of the scriptsig is required due to a bug in OP_CHECKMULTISIG.

The m and n specify the m-of-n signers necessary to authorize a spend.

### P2SH 

Pubkey script: ```OP_HASH160 Hash160(redeemScript) OP_EQUAL```  
Signature script: ```sig (sig) (sig...) redeemScript```

For example, you can do multisig inside a P2SH address:

Pubkey script: ```OP_HASH160 Hash160(redeemScript) OP_EQUAL```  
Redeem script: ```OP_2 A_pubkey B_pubkey C_pubkey OP_3 OP_CHECKMULTISIG```  
Signature script: ```OP_0 A_sig C_sig redeemScript```  

Mention segwit here.
TODO: Segwit notebook.
TODO: Lightning notebook.